In [ ]:
__author__ = 'Benjamin Weaver <benjamin.weaver@noirlab.edu>' # single string; emails in <>
__version__ = '20220406' # yyyymmdd; version datestamp of this notebook
__datasets__ = ['ls_dr8', 'ls_dr9']  # datasets used in this notebook; for available datasets, see cell "Available datasets in Data Lab" further below
__keywords__ = ['query', 'tutorial', 'vospace'] # keywords relevant to this notebook, e.g. ['science case','dwarf galaxies'] Use only keywords from the master list: https://github.com/noaodatalab/notebooks-latest/blob/master/internal/keywords.txt

# Test Legacy Survey Data Releases

*Benjamin Weaver & Astro Data Lab Team*

### Table of Contents

* [Introduction](#Introduction)
* [Imports and Setup](#Imports-and-Setup)
* [File Service](#File-Service)
* [Test Correspondence With Tractor Files](#Test-Correspondence-With-Tractor-Files)
* [Check Photo-z Files](#Check-Photo-z-Files)
* [Check Forced Photometry Files](#Check-Forced-Photometry-Files)
* [QA on ls_id](#QA-on-ls_id)
* [Numerology](#Numerology)

## Introduction

This notebooks provides a series of QA tests on Legacy Survey data releases.  The primary focus is on LS DR8 and DR9, and this notebook can be used with both by setting the `release` variable in the [Global Parameters](#Global-Parameters).

## Imports and Setup

### Imports

In [1]:
%matplotlib inline
from io import BytesIO
import random
from html.parser import HTMLParser
from collections import OrderedDict
import requests
import numpy as np
from astropy.io import fits
from astropy.io.votable import parse_single_table
from astropy.table import Table, join
from dl import queryClient as qc, storeClient as sc, authClient as ac
from dl.helpers.utils import vospace_readable_fileobj

### Other Setup

This can be used to adjust the database location, *e.g.* development or productions.

In [2]:
qc.set_profile('db01')

### Global Parameters

In [3]:
release = 'dr8'
sweep = {'dr8': '8.0', 'dr9': '9.0'}  # Sweep file versions
sdss = {'dr8': 'dr14', 'dr9': 'dr16'}  # SDSS release cross-match
regions = ('north', 'south')  # This will be used a lot.
region_to_release = {'dr8': {'north': (8001,), 'south': (8000,)},
                     'dr9': {'north': (9011,), 'south': (9010, 9012)}}
obj_types = {'dr8': ('objs', 'psf', 'rex', 'exp', 'dev', 'comp'),  # morphological classifications
             'dr9': ('objs', 'psf', 'rex', 'exp', 'dev', 'ser')}
atol = 1.0e-6  # Absolute tolerance in numpy.allclose().
samples = 10  # Number of bricks in each region to test.

### Check the profile 

In [4]:
qc.get_profile()

'db01'

In [5]:
qc.get_svc_url()

'https://datalab.noirlab.edu/query'

In [6]:
if ac.whoAmI() == 'anonymous':
    from getpass import getpass
    token = ac.login(input('Please enter your username: '), getpass('Please enter your password: '))
    print(ac.whoAmI(), token)
else:
    print(ac.whoAmI())

Please enter your username:  davalfher
Please enter your password:  ·········


davalfher davalfher.3437.3437.$1$2tjBBPGk$Iqt4GcjtwkakOPjitv9Db/


## File Service

### List Services

First, make sure the service is listed.

In [7]:
print(sc.services())


                    name   svc   description
                --------   ----  --------
                  ls_dr7   vos   DECam Legacy Survey DR7
                  ls_dr8   vos   DECam Legacy Survey DR8
                 chandra   vos   ChaMPlane: Measuring the Faint X-ray Bin ...
             cosmic_dawn   vos   Cosmic DAWN survey
               deeprange   vos   Deeprange Survey
           deep_ecliptic   vos   Deep Ecliptic Survey
                 des_dr2   vos   Dark Energy Survey DR2
                desi_ets   vos   DESI Early Target Selection
                     dls   vos   Deep Lens Survey
                  flamex   vos   FLAMINGOS Extragalactic Survey
                     fls   vos   First Look Survey
                    fsvs   vos   Faint Sky Variability Survey
               ir_bootes   vos   Infrared Bootes Imaging Survey
                     lgs   vos   Local Group Survey
             gogreen_dr1   vos   GOGREEN DR1 Survey
                     lmc   vos   SuperMACHO Survey
 

### Top-level

Display the top-level files.

In [8]:
sc.ls(f'ls_{release}://').split(',')

['calib',
 'ccds-annotated-90prime-dr8.fits.gz',
 'ccds-annotated-decam-dr8.fits.gz',
 'ccds-annotated-mosaic-dr8.fits.gz',
 'forced',
 'gallery',
 'legacysurvey_dr8.sha256sum',
 'north',
 'randoms',
 'south',
 'survey-bricks.fits.gz',
 'survey-ccds-90prime-dr8.fits.gz',
 'survey-ccds-90prime-dr8.kd.fits',
 'survey-ccds-decam-dr8.fits.gz',
 'survey-ccds-decam-dr8.kd.fits',
 'survey-ccds-mosaic-dr8.fits.gz',
 'survey-ccds-mosaic-dr8.kd.fits']

### Tractor Files

Display random tractor files, just to make sure they are actually present in the file service.

In [11]:
for region in regions:
    print(sc.ls(f'ls_{release}://{region}/tractor/017/'))

legacysurvey_dr8_north_tractor_017.sha256sum,tractor-0170p292.fits,tractor-0170p295.fits,tractor-0171p297.fits,tractor-0171p300.fits,tractor-0173p292.fits,tractor-0173p295.fits,tractor-0173p297.fits,tractor-0174p300.fits
legacysurvey_dr8_south_tractor_017.sha256sum,tractor-0170m077.fits,tractor-0170m080.fits,tractor-0170m082.fits,tractor-0170m085.fits,tractor-0170m087.fits,tractor-0170m090.fits,tractor-0170m092.fits,tractor-0170m095.fits,tractor-0170m122.fits,tractor-0170m125.fits,tractor-0170m127.fits,tractor-0170m130.fits,tractor-0170m132.fits,tractor-0170m135.fits,tractor-0170m160.fits,tractor-0170m162.fits,tractor-0170m165.fits,tractor-0170m167.fits,tractor-0170m190.fits,tractor-0170m192.fits,tractor-0170m195.fits,tractor-0170m197.fits,tractor-0170m212.fits,tractor-0170m215.fits,tractor-0170m217.fits,tractor-0170m220.fits,tractor-0170m235.fits,tractor-0170m237.fits,tractor-0170m240.fits,tractor-0170m242.fits,tractor-0170m255.fits,tractor-0170m257.fits,tractor-0170m260.fits,tractor-

### Sweep

Display sweep files, just to make sure they are actually present in the file service.

In [12]:
for region in regions:
    print(sc.ls(f'ls_{release}://{region}/sweep/{sweep[release]}/'))

legacysurvey_dr8_north_sweep_8.0.sha256sum,sweep-000m005-010p000.fits,sweep-000p000-010p005.fits,sweep-000p040-010p045.fits,sweep-010m005-020p000.fits,sweep-010p000-020p005.fits,sweep-010p025-020p030.fits,sweep-010p030-020p035.fits,sweep-010p035-020p040.fits,sweep-010p040-020p045.fits,sweep-020p025-030p030.fits,sweep-030m005-040p000.fits,sweep-030p000-040p005.fits,sweep-030p025-040p030.fits,sweep-040m005-050p000.fits,sweep-040p000-050p005.fits,sweep-060p065-070p070.fits,sweep-080p055-090p060.fits,sweep-080p060-090p065.fits,sweep-080p065-090p070.fits,sweep-080p075-090p080.fits,sweep-090p030-100p035.fits,sweep-090p035-100p040.fits,sweep-090p040-100p045.fits,sweep-090p045-100p050.fits,sweep-090p050-100p055.fits,sweep-090p055-100p060.fits,sweep-090p060-100p065.fits,sweep-090p065-100p070.fits,sweep-090p070-100p075.fits,sweep-090p075-100p080.fits,sweep-090p080-100p085.fits,sweep-100p025-110p030.fits,sweep-100p030-110p035.fits,sweep-100p035-110p040.fits,sweep-100p040-110p045.fits,sweep-100p04

## Test Correspondence With Tractor Files

If the data are loaded successfully, the values in the database should match the values in the original Tractor files.  This section selects a random subset of the bricks in each region and compares the data for that brick.

The set of columns below is motivated by the 2021-2022 reload of `ls_dr8` due to column ordering problems.

In [13]:
columns = ['objid', 'brickname', 'flux_g', 'flux_ivar_g', 'flux_r', 'flux_ivar_r', 'flux_z', 'flux_ivar_z',
           'flux_w1', 'flux_ivar_w1', 'flux_w2', 'flux_ivar_w2', 'flux_w3', 'flux_ivar_w3', 'flux_w4', 'flux_ivar_w4',
           'maskbits', 'gaia_astrometric_n_obs_al', 'gaia_astrometric_n_good_obs_al', 'gaia_phot_variable_flag', 'gaia_duplicated_source',
           'gaia_phot_g_n_obs', 'gaia_phot_bp_n_obs', 'gaia_phot_rp_n_obs']
if release == 'dr8':
    columns += ['brightblob', 'gaia_pointsource']
col = ', '.join(columns)
for region in regions:
    q0 = f"""SELECT b.brickid, bb.brickname FROM ls_{release}.bricks_{region[0]} AS bb
JOIN ls_{release}.bricks AS b
ON bb.brickname = b.brickname
ORDER by b.brickid
"""
    response = qc.query(sql=q0, fmt='votable', timeout=600)
    # print(response)
    bricks_data = parse_single_table(BytesIO(response.encode('utf-8'))).to_table()
    sample = random.sample(range(len(bricks_data)), samples)
    for i in sample:
        brickname = bricks_data['brickname'][i]
        brickid = bricks_data['brickid'][i]
        print(f"region = '{region}'; brickname = '{brickname}'; brickid = {brickid:d}")
        tractor = f'ls_{release}://{region}/tractor/{brickname[0:3]}/tractor-{brickname}.fits'
        tractor_data = Table.read(sc.get(tractor, mode='fileobj'), format='fits')
        q = f"""SELECT {col} FROM ls_{release}.tractor_{region[0]} WHERE brickid = {brickid:d} ORDER BY objid"""
        response = qc.query(sql=q, fmt='votable', timeout=600)
        try:
            db_data = parse_single_table(BytesIO(response.encode('utf-8'))).to_table()
        except ValueError:
            print(response)
        assert (db_data['brickname'] == brickname).all()
        assert (tractor_data['objid'] == np.arange(len(tractor_data))).all()
        assert len(db_data) == tractor_data['brick_primary'].sum()
        primary = np.nonzero(tractor_data['brick_primary'])[0]
        assert (db_data['objid'] == tractor_data['objid'][primary]).all()
        assert np.allclose(db_data['flux_g'], tractor_data['flux_g'][primary], atol=atol)
        assert np.allclose(db_data['flux_r'], tractor_data['flux_r'][primary], atol=atol)
        assert np.allclose(db_data['flux_z'], tractor_data['flux_z'][primary], atol=atol)
        assert np.allclose(db_data['flux_w1'], tractor_data['flux_w1'][primary], atol=atol)
        assert np.allclose(db_data['flux_w2'], tractor_data['flux_w2'][primary], atol=atol)
        assert np.allclose(db_data['flux_w3'], tractor_data['flux_w3'][primary], atol=atol)
        assert np.allclose(db_data['flux_w4'], tractor_data['flux_w4'][primary], atol=atol)
        assert np.allclose(db_data['flux_ivar_g'], tractor_data['flux_ivar_g'][primary], atol=atol)
        assert np.allclose(db_data['flux_ivar_r'], tractor_data['flux_ivar_r'][primary], atol=atol)
        assert np.allclose(db_data['flux_ivar_z'], tractor_data['flux_ivar_z'][primary], atol=atol)
        assert np.allclose(db_data['flux_ivar_w1'], tractor_data['flux_ivar_w1'][primary], atol=atol)
        assert np.allclose(db_data['flux_ivar_w2'], tractor_data['flux_ivar_w2'][primary], atol=atol)
        assert np.allclose(db_data['flux_ivar_w3'], tractor_data['flux_ivar_w3'][primary], atol=atol)
        assert np.allclose(db_data['flux_ivar_w4'], tractor_data['flux_ivar_w4'][primary], atol=atol)
        if 'brightblob' in columns:
            assert (db_data['brightblob'] == tractor_data['brightblob'][primary]).all()
        assert (db_data['maskbits'] == tractor_data['maskbits'][primary]).all()
        if 'gaia_pointsource' in columns:
            assert (db_data['gaia_pointsource'] == tractor_data['gaia_pointsource'][primary]).all()
        assert (db_data['gaia_astrometric_n_obs_al'] == tractor_data['gaia_astrometric_n_obs_al'][primary]).all()
        assert (db_data['gaia_astrometric_n_good_obs_al'] == tractor_data['gaia_astrometric_n_good_obs_al'][primary]).all()
        assert (db_data['gaia_phot_variable_flag'] == tractor_data['gaia_phot_variable_flag'][primary]).all()
        assert (db_data['gaia_duplicated_source'] == tractor_data['gaia_duplicated_source'][primary]).all()
        assert (db_data['gaia_phot_g_n_obs'] == tractor_data['gaia_phot_g_n_obs'][primary]).all()
        assert (db_data['gaia_phot_bp_n_obs'] == tractor_data['gaia_phot_bp_n_obs'][primary]).all()
        assert (db_data['gaia_phot_rp_n_obs'] == tractor_data['gaia_phot_rp_n_obs'][primary]).all()

region = 'north'; brickname = '1924p780'; brickid = 654778
region = 'north'; brickname = '2031p517'; brickid = 590794
region = 'north'; brickname = '1447p842'; brickid = 660407
region = 'north'; brickname = '2268p435'; brickid = 558762
region = 'north'; brickname = '2509p602'; brickid = 618356
region = 'north'; brickname = '2243p610'; brickid = 620431
region = 'north'; brickname = '2381p590'; brickid = 614675
region = 'north'; brickname = '1575p662'; brickid = 633819
region = 'north'; brickname = '2798p380'; brickid = 534882
region = 'north'; brickname = '2310p675'; brickid = 636779
region = 'south'; brickname = '3387m115'; brickid = 265860
region = 'south'; brickname = '1612p225'; brickid = 457452
region = 'south'; brickname = '3147m097'; brickid = 275687
region = 'south'; brickname = '3236m672'; brickid = 26255
region = 'south'; brickname = '3210p125'; brickid = 403129
region = 'south'; brickname = '0174m577'; brickid = 51065
region = 'south'; brickname = '1065p325'; brickid = 508394

## Check Photo-z Files

Just make sure the files are actually present in the file service.

In [14]:
for region in regions:
    print(sc.ls(f'ls_{release}://{region}/sweep'))
    print(sc.ls(f'ls_{release}://{region}/sweep/{sweep[release]}-photo-z/'))

8.0,8.0-photo-z
legacysurvey_dr8_north_sweep_8.0-photo-z.sha256sum,sweep-000m005-010p000-pz.fits,sweep-000p000-010p005-pz.fits,sweep-000p040-010p045-pz.fits,sweep-010m005-020p000-pz.fits,sweep-010p000-020p005-pz.fits,sweep-010p025-020p030-pz.fits,sweep-010p030-020p035-pz.fits,sweep-010p035-020p040-pz.fits,sweep-010p040-020p045-pz.fits,sweep-020p025-030p030-pz.fits,sweep-030m005-040p000-pz.fits,sweep-030p000-040p005-pz.fits,sweep-030p025-040p030-pz.fits,sweep-040m005-050p000-pz.fits,sweep-040p000-050p005-pz.fits,sweep-060p065-070p070-pz.fits,sweep-080p055-090p060-pz.fits,sweep-080p060-090p065-pz.fits,sweep-080p065-090p070-pz.fits,sweep-080p075-090p080-pz.fits,sweep-090p030-100p035-pz.fits,sweep-090p035-100p040-pz.fits,sweep-090p040-100p045-pz.fits,sweep-090p045-100p050-pz.fits,sweep-090p050-100p055-pz.fits,sweep-090p055-100p060-pz.fits,sweep-090p060-100p065-pz.fits,sweep-090p065-100p070-pz.fits,sweep-090p070-100p075-pz.fits,sweep-090p075-100p080-pz.fits,sweep-090p080-100p085-pz.fits,swe

## Check Forced Photometry Files

Just make sure the files are actually present in the file service.

In [15]:
if release == 'dr8':
    print(sc.ls(f'ls_{release}://forced/'))
    for camera in ('90prime', 'decam', 'mosaic'):
        print(sc.ls(f'ls_{release}://forced/{camera}'))
else:
    for region in regions:
        print(sc.ls(f'ls_{release}://{region}/forced-ccd/'))
        if region == 'north':
            print(sc.ls(f'ls_{release}://{region}/forced-ccd/90prime'))
            print(sc.ls(f'ls_{release}://{region}/forced-ccd/mosaic'))
        else:
            print(sc.ls(f'ls_{release}://{region}/forced-ccd/decam'))

90prime,decam,mosaic
70680,73380,73390,73900,73910,73990,74000,74010,74020,74030,74040,74050,74170,74210,74220,74230,74240,74250,74270,74280,74290,74300,74310,74320,74330,74340,74350,74510,74520,74540,74560,74570,74580,74590,74600,74610,74800,74810,74820,74830,74840,74870,74880,74900,74910,74920,75060,75070,75080,75090,75100,75110,75120,75130,75140,75160,75170,75180,75190,75200,75210,75220,75240,75350,75360,75370,75380,75390,75400,75410,75420,75430,75440,75450,75480,75500,75510,75660,75670,75710,75720,75730,75740,75750,75760,75770,75780,75790,75800,75810,75820,76980,76990,77720,77760,77790,77800,77810,77820,77830,77840,77850,77860,77870,77880,77890,77900,77980,78050,78060,78070,78080,78090,78100,78110,78130,78140,78150,78160,78170,78180,78190,78300,78310,78320,78330,78340,78350,78360,78370,78380,78390,78400,78410,78420,78440,78450,78460,78470,78480,78610,78620,78630,78640,78650,78660,78670,78690,78700,78720,78730,78740,78880,78890,78900,78910,78920,78930,78940,78950,78960,78970,78990,7

## Test Joins on specobjid

In DR8, a pre-computed match table generated by the Legacy Survey collaboration was loaded.  In DR9 we use the crossmatch table generated by the Data Lab team instead.

In [16]:
if release == 'dr8':
    for region in regions:
        q = f"""SELECT ll.{sdss[release]}_specobjid, ll.plate, ll.fiberid, ll.mjd, s.specobjid, s.plate, s.fiberid, s.mjd
FROM
    (SELECT l.*,
        sdss_{sdss[release]}.specobjid(CAST(l.plate AS smallint), CAST(l.fiberid AS smallint), l.mjd, l.run2d)
        AS {sdss[release]}_specobjid
    FROM ls_{release}.{release}_{region}_specobj_{sdss[release]} AS l) AS ll
JOIN sdss_{sdss[release]}.specobjall AS s ON ll.{sdss[release]}_specobjid = s.specobjid
LIMIT 200"""
        response = qc.query(sql=q, fmt='csv', timeout=600)
        print(response)
else:
    q = f"""SELECT t.ls_id, t.ra, t.dec, t.mag_g, t.mag_r, t.mag_z, s.specobjid, s.plate, s.fiberid, s.mjd
FROM
    (SELECT id1 AS ls_id, id2 AS specobjid FROM ls_{release}.x1p5__tractor__sdss_{sdss[release]}__specobj LIMIT 200) AS ll
JOIN ls_{release}.tractor AS t ON ll.ls_id = t.ls_id
JOIN sdss_{sdss[release]}.specobj AS s ON ll.specobjid = s.specobjid
"""
    response = qc.query(sql=q, fmt='csv', timeout=600)
    print(response)

dr14_specobjid,plate,fiberid,mjd,specobjid,plate,fiberid,mjd
299511942555396096,266,82,51630,299511942555396096,266,82,51630
299512217433303040,266,83,51630,299512217433303040,266,83,51630
299512492311209984,266,84,51630,299512492311209984,266,84,51630
299513591822837760,266,88,51630,299513591822837760,266,88,51630
299513866700744704,266,89,51630,299513866700744704,266,89,51630
299514141578651648,266,90,51630,299514141578651648,266,90,51630
299514416456558592,266,91,51630,299514416456558592,266,91,51630
299514691334465536,266,92,51630,299514691334465536,266,92,51630
299516890357721088,266,100,51630,299516890357721088,266,100,51630
299518264747255808,266,105,51630,299518264747255808,266,105,51630
299522662793766912,266,121,51630,299522662793766912,266,121,51630
299523487427487744,266,124,51630,299523487427487744,266,124,51630
299524312061208576,266,127,51630,299524312061208576,266,127,51630
299526511084464128,266,135,51630,299526511084464128,266,135,51630
299527610596091904,266,139,5163

## QA on ls_id

Check that `ls_id` is set correctly on the tractor region tables as well as the joined tractor table.

All objects should have `brick_primary == 1`.  First test the north and south tables.

In [21]:
for region in regions:
    q = f"SELECT brick_primary FROM ls_{release}.tractor_{region[0]} WHERE brick_primary != 1"
    response = qc.query(sql=q, async_=False, wait=True, poll=60, timeout=86400, verbose=True)
    print(response)

brick_primary

brick_primary



Now test the joined table. Use async query because this may time out otherwise.

In [20]:
q = f"SELECT brick_primary FROM ls_{release}.tractor WHERE brick_primary != 1"
response = qc.query(sql=q, async_=True, wait=True, poll=60, timeout=86400, verbose=True)
print(response)

EXECUTING
Status = EXECUTING; elapsed time: 60, timeout in 86340
EXECUTING
Status = EXECUTING; elapsed time: 120, timeout in 86280
EXECUTING
Status = EXECUTING; elapsed time: 180, timeout in 86220
EXECUTING
Status = EXECUTING; elapsed time: 240, timeout in 86160
EXECUTING
Status = EXECUTING; elapsed time: 300, timeout in 86100
EXECUTING
Status = EXECUTING; elapsed time: 360, timeout in 86040
EXECUTING
Status = EXECUTING; elapsed time: 420, timeout in 85980
EXECUTING
Status = EXECUTING; elapsed time: 480, timeout in 85920
EXECUTING
Status = EXECUTING; elapsed time: 540, timeout in 85860
EXECUTING
Status = EXECUTING; elapsed time: 600, timeout in 85800
EXECUTING
Status = EXECUTING; elapsed time: 660, timeout in 85740
EXECUTING
Status = EXECUTING; elapsed time: 720, timeout in 85680
EXECUTING
Status = EXECUTING; elapsed time: 780, timeout in 85620
EXECUTING
Status = EXECUTING; elapsed time: 840, timeout in 85560
EXECUTING
Status = EXECUTING; elapsed time: 900, timeout in 85500
EXECUTING
S

Now check the actual `ls_id`.

In [22]:
for region in regions:
    q = f"SELECT ls_id, release, brickid, objid FROM ls_{release}.tractor_{region[0]} LIMIT 100000"
    response = qc.query(sql=q, fmt='csv', timeout=600)
    a, b, c, d = zip(*[tuple(map(int, l.strip().split(','))) for l in response.strip().split('\n') if not l.startswith('ls_id')])
    ls_id = np.array(a, dtype=np.uint64)
    ls_release = np.array(b, dtype=np.uint64)
    brickid = np.array(c, dtype=np.uint64)
    objid = np.array(d, dtype=np.uint64)
    assert ((((ls_release << 40) | (brickid << 16) | (objid)) - ls_id) == 0).all()

In [ ]:
q = f"SELECT ls_id, release, brickid, objid FROM ls_{release}.tractor LIMIT 100000"
response = qc.query(sql=q, fmt='csv', timeout=600)
a, b, c, d = zip(*[tuple(map(int, l.strip().split(','))) for l in response.strip().split('\n') if not l.startswith('ls_id')])
ls_id = np.array(a, dtype=np.uint64)
ls_release = np.array(b, dtype=np.uint64)
brickid = np.array(c, dtype=np.uint64)
objid = np.array(d, dtype=np.uint64)
assert ((((ls_release << 40) | (brickid << 16) | (objid)) - ls_id) == 0).all()

### QA on ls_id in Forced Photometry

This tests joins of the forced photometry data with the main tractor table.

In [ ]:
for region in regions:
    q0 = f"""SELECT ls_id, release, brickid, objid,
    (CAST(release AS bigint) << 40) | 
    (CAST(brickid AS bigint) << 16) | 
    (CAST(objid AS bigint)) AS test_ls_id
FROM ls_{release}.tractor_{region[0]} LIMIT 10"""
    response = qc.query(sql=q0, fmt='csv', timeout=600)
    ls_id = ', '.join([l.strip().split(',')[0] for l in response.strip().split('\n') if not l.startswith('ls_id')])
    q = f"""SELECT * FROM ls_{release}.tractor_{region[0]} AS t
JOIN ls_{release}.forced AS f ON t.ls_id = f.ls_id
JOIN ls_{release}.ccds_annotated AS c ON c.ls_ccd_id = f.ls_ccd_id
WHERE t.ls_id IN ({ls_id})
"""
    print(q)
    response = qc.query(sql=q, format='csv', timeout=600)
    print(len(response.strip().split('\n')[1:]))
    print(response)

## Numerology

### Source Table Integrity Check

Check that the [Morphological Classification table](https://www.legacysurvey.org/dr8/description/#morphological-classification) is self-consistent.  This table can then be used to check the numbers of different types in the database.

In [ ]:
class LSDescriptionHTMLParser(HTMLParser):
    """Parse!
    """
    in_section = False
    in_table = False
    in_thead = False
    in_tbody = False
    in_data_row = False
    get_header = False
    get_data = False
    data = [['type', 'all_north', 'all_south', 'resolved_north', 'resolved_south', 'unique'],]

    def handle_starttag(self, tag, attrs):
        a = OrderedDict(attrs)
        if tag == 'div' and 'id' in a and a['id'] == 'morphological-classification':
            self.in_section = True
        if tag == 'table' and self.in_section:
            self.in_table = True
        if tag == 'thead' and self.in_table:
            self.in_thead = True
        if tag == 'tbody' and self.in_table:
            self.in_tbody = True
        if tag == 'th' and self.in_thead:
            self.get_header = True
        if tag == 'tr' and self.in_tbody:
            self.in_data_row = True
        if tag == 'td' and self.in_tbody:
            self.get_data = True
            

    def handle_endtag(self, tag):
        if tag == 'div' and self.in_section:
            self.in_section = False
        if tag == 'table' and self.in_table:
            self.in_table = False
        if tag == 'thead' and self.in_table:
            self.in_thead = False
        if tag == 'tbody' and self.in_table:
            self.in_tbody = False
        if tag == 'th' and self.get_header:
            self.get_header = False
        if tag == 'tr' and self.in_data_row:
            self.in_data_row = False
        if tag == 'td' and self.get_data:
            self.get_data = False

    def handle_data(self, data):
        if self.in_table:
            if self.in_thead:
                if self.get_header:
                    # print(data)
                    pass
            if self.in_tbody:
                if self.in_data_row and self.get_data:
                    try:
                        i = int(data.replace(',', ''))
                    except ValueError:
                        self.data.append([data,])
                    else:
                        self.data[-1].append(i)


r = requests.get(f'https://www.legacysurvey.org/{release}/description/')
parser = LSDescriptionHTMLParser()
parser.feed(r.text)
morph_table = Table(list(zip(*parser.data[1:])), names=parser.data[0])

In [ ]:
morph_table

In [ ]:
assert (morph_table['resolved_north'] + morph_table['resolved_south'] == morph_table['unique']).all()

In [ ]:
assert morph_table['all_north'][1:].sum() == morph_table['all_north'][0]
assert morph_table['all_south'][1:].sum() == morph_table['all_south'][0]
assert morph_table['resolved_north'][1:].sum() == morph_table['resolved_north'][0]
assert morph_table['resolved_south'][1:].sum() == morph_table['resolved_south'][0]
assert morph_table['unique'][1:].sum() == morph_table['unique'][0]
assert (morph_table['all_north'][0] - morph_table['all_north'][1:6].sum()) == morph_table['all_north'][-1]

### Compare Source Table to survey-brick files

In `ls_dr8` there is a [known overflow issue](https://www.legacysurvey.org/dr8/issues/#overflow-in-survey-bricks-dr8-south-fits-gz) that will produce -65536 instead of 0.

In [ ]:
for region in regions:
    survey_bricks = Table.read(sc.get(f'ls_{release}://{region}/survey-bricks-{release}-{region}.fits.gz', mode='fileobj'), format='fits')
    select_objs = ', '.join(["SUM(n{0}) AS n{0}".format(o) for o in obj_types[release]])
    q = f"""SELECT {select_objs} FROM ls_{release}.bricks_{region[0]}"""
    print(q)
    response = qc.query(sql=q, fmt='csv', timeout=600)
    database_bricks = np.array([int(x) for x in response.strip().split('\n')[1].split(',')])
    ndup = 0
    for k, t in enumerate(obj_types[release]):
        try:
            assert survey_bricks['n'+t].sum() == morph_table[f'all_{region}'][k]
        except AssertionError:
            print('FILE:', region, t.upper(), survey_bricks['n'+t].sum() - morph_table[f'all_{region}'][k])
        try:
            assert database_bricks[k] == morph_table[f'all_{region}'][k]
        except AssertionError:
            print('DB:', region, t.upper(), database_bricks[k] - morph_table[f'all_{region}'][k])
        if k == 0:
            ndup = survey_bricks['n'+t].sum()
        else:
            ndup -= survey_bricks['n'+t].sum()
    assert ndup == morph_table[f'all_{region}'][-1]
    assert survey_bricks['nsimp'].sum() == 0

### Number of Rows in the Table

This is where there is a potential inconsistency for `ls_dr8`.  

In [ ]:
for region in regions:
    q = f"SELECT COUNT(*) AS n_rows FROM ls_{release}.tractor_{region[0]}"
    # response_psql = !psql --no-align --field-separator=, --tuples-only --username=dlquery --host=db01.datalab.noirlab.edu --dbname=tapdb --command="$q"
    response_async = qc.query(sql=q, fmt='csv', async_=True, wait=True, poll=60, timeout=86400, verbose=True)
    # print(response_psql.nlstr)
    print(response_async)
    # print(morph_table[f'all_{region}'][0] - int(response_psql.nlstr.strip().split('\n')[0]))
    print(morph_table[f'all_{region}'][0] - int(response_async.strip().split('\n')[1]))

In [ ]:
q = f"SELECT COUNT(*) AS n_rows FROM ls_{release}.tractor"
response = qc.query(sql=q, fmt='csv', async_=True, wait=True, poll=60, timeout=86400, verbose=True)
print(response)
print(morph_table['unique'][0] - int(response.strip().split('\n')[1]))

### Number of Distinct Objects

Here we compare the number of objects of a given type to the morphology table obtained above.

In [ ]:
for region in regions:
    q = f"SELECT type, COUNT(type) AS n_type FROM ls_{release}.tractor_{region[0]} GROUP BY type"
    # response_psql = !psql --no-align --field-separator=, --tuples-only --username=dlquery --host=db01.datalab.noirlab.edu --dbname=tapdb --command="$q"
    response_async = qc.query(sql=q, fmt='csv', async_=True, wait=True, poll=60, timeout=86400, verbose=True)
    # print(response_psql.nlstr)
    print(response_async)
    total_types = 0
    for key, val in dict([x.split(',') for x in response_async.strip().split('\n')[1:]]).items():
        try:
            assert morph_table[morph_table['type'] == key][f'all_{region}'] == int(val)
        except AssertionError:
            print(region, key, morph_table[morph_table['type'] == key][f'all_{region}'] - int(val))
        total_types += int(val)
    if morph_table[f'all_{region}'][0] != total_types:
        print(region, total_types, morph_table[f'all_{region}'][0] - total_types)

In [ ]:
q = f"SELECT type, COUNT(type) AS n_type FROM ls_{release}.tractor GROUP BY type"
# response_psql = !psql --no-align --field-separator=, --tuples-only --username=dlquery --host=db01.datalab.noirlab.edu --dbname=tapdb --command="$q"
response_async = qc.query(sql=q, fmt='csv', async_=True, wait=True, poll=60, timeout=86400, verbose=True)
# print(response_psql.nlstr)
print(response_async)
total_types = 0
for key, val in dict([x.split(',') for x in response_async.strip().split('\n')[1:]]).items():
    try:
        assert morph_table[morph_table['type'] == key]['unique'] == int(val)
    except AssertionError:
        print(region, key, morph_table[morph_table['type'] == key]['unique'] - int(val))
    total_types += int(val)
if morph_table['unique'][0] != total_types:
    print(region, total_types, morph_table['unique'][0] - total_types)

### QA on release Column

Ensure that the release column matches the region of the table.  Use async queries to avoid timeouts.

In [ ]:
for region in regions:
    r = ', '.join(map(str, region_to_release[release][region]))
    q = f"SELECT COUNT(*) AS n FROM ls_{release}.tractor_{region[0]} WHERE release NOT IN ({r})"
    print(q)
    response = qc.query(sql=q, fmt='csv', async_=True, wait=True, poll=60, timeout=86400, verbose=True)
    print(response)

### Check Construction of Full tractor Table

If the full tractor table is constructed properly, the region `dec >= 32.375 AND glat > 0` will contain only objects with a "north" `release`.

In [ ]:
q = f"SELECT release, COUNT(release) AS n FROM ls_{release}.tractor WHERE dec >= 32.375 AND glat > 0 GROUP BY release"
response = qc.query(sql=q, fmt='csv', async_=True, wait=True, poll=60, timeout=86400, verbose=True)
print(response)

And the region `dec < 32.375 OR glat < 0` should contain only objects with a "south" `release`.

In [ ]:
q = f"SELECT release, COUNT(release) AS n FROM ls_{release}.tractor WHERE dec < 32.375 OR glat < 0 GROUP BY release"
response = qc.query(sql=q, fmt='csv', async_=True, wait=True, poll=60, timeout=86400, verbose=True)
print(response)

### Number of Bricks

Is the number of distinct bricks in the `tractor_(n|s)` tables the same as the number of bricks in the `bricks_(n|s)` tables? In theory, some bricks may have no objects with `brick_primary == 1` but are still listed in `bricks_(n|s)`.

In [ ]:
for region in regions:
    q = f"SELECT DISTINCT b.brickid FROM ls_{release}.bricks_{region[0]} AS br JOIN ls_{release}.bricks AS b ON br.brickname = b.brickname WHERE br.nobjs > 0"
    response = qc.query(sql=q, fmt='csv', timeout=600)
    all_brickids = response.strip().split('\n')[1:]
    print(len(all_brickids))
    q = f"SELECT DISTINCT brickid FROM ls_{release}.tractor_{region[0]}"
    response = qc.query(sql=q, fmt='csv', timeout=600)
    tractor_brickids = response.strip().split('\n')[1:]
    print(len(tractor_brickids))
    missing_brickids = list()
    for b in all_brickids:
        if b not in tractor_brickids:
            missing_brickids.append(b)
    if len(missing_brickids) > 0:
        join_brickids = ', '.join(missing_brickids)
        q = f"SELECT b.brickname, br.nobjs FROM ls_{release}.bricks_{region[0]} AS br JOIN ls_{release}.bricks AS b ON br.brickname = b.brickname WHERE b.brickid IN ({join_brickids})"
        print(q)
        response = qc.query(sql=q, fmt='csv', timeout=600)
        missing_bricknames = dict([x.split(',') for x in response.strip().split('\n')[1:]])
        print(missing_bricknames)
        missing_nobjs = 0
        for m in missing_bricknames:
            print(sc.ls(f'ls_{release}://{region}/tractor/{m[0:3]}/tractor-{m}.fits'))
            missing_nobjs += int(missing_bricknames[m])
        print(missing_nobjs)

#### Breakdown numbers by brick

Brick `2296p020` in `ls_dr8` in the south is a [known issue](https://www.legacysurvey.org/dr8/issues/#overflow-in-survey-bricks-dr8-south-fits-gz).

In [ ]:
for region in regions:
    q = f"SELECT brickname, nobjs FROM ls_{release}.bricks_{region[0]} WHERE nobjs > 0 ORDER BY brickname"
    response = qc.query(sql=q, fmt='csv', timeout=600)
    objects_per_brick = dict([x.split(',') for x in response.strip().split('\n')[1:]])
    q = f"SELECT brickname, COUNT(*) FROM ls_{release}.tractor_{region[0]} GROUP BY brickname ORDER BY brickname"
    response = qc.query(sql=q, fmt='csv', async_=True, wait=True, poll=60, timeout=86400, verbose=True)
    objects_per_brick_in_tractor = dict([x.split(',') for x in response.strip().split('\n')[1:]])
    for b in objects_per_brick:
        if b in objects_per_brick_in_tractor:
            if objects_per_brick[b] !=  objects_per_brick_in_tractor[b]:
                print(f"{b}: {objects_per_brick[b]} (ls_{release}.bricks_{region[0]}) != {objects_per_brick_in_tractor[b]} (ls_{release}.tractor_{region[0]})")
        else:
            print(f"{b} is not in ls_{release}.tractor_{region[0]}!")
    for b in objects_per_brick_in_tractor:
        if b not in objects_per_brick:
            print(f"{b} is not in ls_{release}.bricks_{region[0]}!")